In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor
from nltk.stem import SnowballStemmer	# 获取词干

In [2]:
# from utils import IOUtils
# df_train=pd.read_csv(IOUtils.get_path_sources("train.csv","home-depot-product-search-relevance"))
# df_test=pd.read_csv(IOUtils.get_path_sources("test.csv","home-depot-product-search-relevance"))
# df_desc=pd.read_csv(IOUtils.get_path_sources("product_descriptions.csv","home-depot-product-search-relevance"))

df_train = pd.read_csv("../sources/home-depot-product-search-relevance/train.csv", encoding="ISO-8859-1")
df_test = pd.read_csv("../sources/home-depot-product-search-relevance/test.csv", encoding="ISO-8859-1")
df_desc = pd.read_csv("../sources/home-depot-product-search-relevance/product_descriptions.csv")

OSError: File b'../sources/home-depot-product-search-relevance/train.csv' does not exist

In [11]:
df_train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [12]:
df_test.head()

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668


In [13]:
df_desc.head()

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...
4,100005,Update your bathroom with the Delta Vero Singl...


In [10]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all.head()

,id,product_title,product_uid,relevance,search_term
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet


In [11]:
df_all=pd.merge(df_all,df_desc,how="left",on="product_uid")
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description
0,2,Simpson Strong-Tie 12-Gauge Angle,100001,3.00,angle bracket,"Not only do angles make joints stronger, they ..."
1,3,Simpson Strong-Tie 12-Gauge Angle,100001,2.50,l bracket,"Not only do angles make joints stronger, they ..."
2,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,100002,3.00,deck over,BEHR Premium Textured DECKOVER is an innovativ...
3,16,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.33,rain shower head,Update your bathroom with the Delta Vero Singl...
4,17,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,100005,2.67,shower only faucet,Update your bathroom with the Delta Vero Singl...


In [12]:
stemmer=SnowballStemmer("english")
# 转化小写并将多个空格转化为1个空格
def str_stemmer(s):
	return " ".join((stemmer.stem(word) for word in s.strip().lower().split()))
# 为了计算 关键词 的有效性,我们可以native地直接看 出现了多少次
def str_common_word(str1,str2):
	return sum(int(str2.find(word)>0) for word in str1.split())

In [14]:
df_all["search_term"]=df_all["search_term"].map(lambda x:str_stemmer(x))
df_all["product_title"]=df_all["product_title"].map(lambda x:str_stemmer(x))
df_all["product_description"]=df_all["product_description"].map(lambda x:str_stemmer(x))
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description
0,2,simpson strong-ti 12-gaug angl,100001,3.00,angl bracket,"not onli do angl make joint stronger, they als..."
1,3,simpson strong-ti 12-gaug angl,100001,2.50,l bracket,"not onli do angl make joint stronger, they als..."
2,9,behr premium textur deckov 1-gal. #sc-141 tugb...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...
3,16,delta vero 1-handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero single...
4,17,delta vero 1-handl shower onli faucet trim kit...,100005,2.67,shower on faucet,updat your bathroom with the delta vero single...
